<a href="https://colab.research.google.com/github/anmaxwell/UniNotebooks/blob/master/2AssesB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scattertext

     |████████████████████████████████| 6.8MB 2.7MB/s 


In [2]:
pip install "git+https://github.com/facebookresearch/fastText.git"

  Cloning https://github.com/facebookresearch/fastText.git to /tmp/pip-req-build-ec8divsm
  Running command git clone -q https://github.com/facebookresearch/fastText.git /tmp/pip-req-build-ec8divsm
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2871352 sha256=356e80f1adb173f03533d5ae9ebd50b9f9d99c3062f886853c04e370d49e5796
  Stored in directory: /tmp/pip-ephem-wheel-cache-vkyfwac2/wheels/69/f8/19/7f0ab407c078795bc9f86e1f6381349254f86fd7d229902355
Successfully built fasttext


Install all necessary packages

In [3]:
import fasttext.util
import numpy as np
import pandas as pd
import re
import scattertext as st
import spacy

from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from scattertext import CorpusFromPandas, produce_scattertext_explorer
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


Read in data and look at first item

In [6]:
df = pd.read_csv('agr_en_train.csv', names=['unique_id','text','aggression-level'], sep=',')
print(df.iloc[0])

unique_id                                 facebook_corpus_msr_1723796
text                Well said sonu..you have courage to stand agai...
aggression-level                                                  OAG
Name: 0, dtype: object


Check for missing values

In [7]:
df.isna().values.any()

False

Count the occurrences per Aggression Level

In [8]:
df['aggression-level'].value_counts() 

NAG    5051
CAG    4240
OAG    2708
Name: aggression-level, dtype: int64

Identify the top occurring words per Aggression Level

In [9]:
nlp = spacy.load('en')
df['parsed'] = df.text.apply(nlp)
data = st.CorpusFromParsedDocuments(df, category_col='aggression-level', 
                                      parsed_col='parsed').build().remove_terms(nlp.Defaults.stop_words, ignore_absences=True)

freq_df = data.get_term_freq_df()
oag_tw = freq_df.sort_values(by=['OAG freq'], ascending=False)
oag_tw = oag_tw.drop(oag_tw.columns[[1,2]], axis=1)
nag_tw = freq_df.sort_values(by=['NAG freq'], ascending=False)
nag_tw = nag_tw.drop(nag_tw.columns[[0,2]], axis=1)
cag_tw = freq_df.sort_values(by=['CAG freq'], ascending=False)
cag_tw = cag_tw.drop(cag_tw.columns[[0,1]], axis=1)

print(oag_tw.head())
print(nag_tw.head())
print(cag_tw.head())

KeyboardInterrupt: ignored

Replace the text labels with values

In [11]:
df['aggression-level'] = df['aggression-level'].replace({ 'OAG' : 0, 'NAG' : 1, 'CAG' : 2 }) 

TypeError: ignored

Clean up the text to remove non-ASCII characters and set all text to lowercase (potential to remove numbers and stopwords)

In [0]:
def clean_up(text):
  
  text = text.replace('&', ' and ')
  text = text.replace('@', ' at ')
  text = re.sub(r'[^\x00-\x7f]',r' ',text)
  text = text.lower().split()

  return text

Load the fasttext model

In [13]:
fasttext.util.download_model('en', if_exists='ignore') 
ft = fasttext.load_model('cc.en.300.bin')

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Set the paramters for making the Embedding Layer

In [0]:
#unique_words = len(set(df['text'].str.cat(sep=' ').lower().split()))
unique_words = 30
data_count = len(df)
dims = ft.get_dimension()

Create the word vectors

In [0]:
def create_vectors(input_text):
    
    v = np.zeros((unique_words, dims))
    input_text = clean_up(input_text)

    for i, word in enumerate(input_text):
        v[i, :] = ft.get_word_vector(word).astype('float32')

    return v

Create Embedding

In [0]:
def create_embeddings(dataframe):

    e = np.zeros((data_count, unique_words, dims), dtype='float32')

    for i, review in enumerate(df['text'].values):
        e[i, :] = create_vectors(review)

    return e


In [0]:
create_embeddings(df['text'])